In [1]:
!which python

/home/user/jfayzullaev/stellar-clustering/.venv-vis/bin/python


In [3]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import normalized_mutual_info_score as NMI, adjusted_rand_score as ARI


In [14]:
LPA_COMM_TX_FILE   = "transaction/lpa_tx_lcc_lpa_communities.csv" 
LPA_COMM_TL_FILE   = "trustline/lpa_trust_lcc_lpa_communities.csv" 


In [19]:
NORM_LABELS_PATH = os.path.expanduser("~/stellar-clustering/network/labled-data/labels/label-normalization/labels_entities_normalized.csv") 

In [16]:
N_SPLITS = 5
RANDOM_STATE = 42

In [8]:
def overall_purity(df_comm_name: pd.DataFrame) -> float:
    if df_comm_name.empty:
        return np.nan
    counts = df_comm_name.groupby(['community', 'name']).size().reset_index(name='cnt')
    totals = counts.groupby('community')['cnt'].sum()
    max_per_comm = counts.groupby('community')['cnt'].max()
    return float(max_per_comm.sum() / totals.sum())

In [ ]:
def load_communities_fixed_resolution(lpa_file: str) -> pd.DataFrame:

    if not os.path.exists(lpa_file):
        raise FileNotFoundError(f"Missing LPA file: {lpa_file}")
    df = pd.read_csv(lpa_file)
    if 'account_id' not in df.columns and 'node' in df.columns:
        df = df.rename(columns={'node': 'account_id'})
    if 'community' not in df.columns:
        if 'lpa_comm' in df.columns:
            df = df.rename(columns={'lpa_comm': 'community'})
        elif 'comm' in df.columns:
            df = df.rename(columns={'comm': 'community'})

    df = df[['account_id', 'community']].dropna().drop_duplicates()

    try:
        df['account_id'] = df['account_id'].astype(int)
    except Exception:
        df['account_id'] = df['account_id'].astype(str)
    return df


In [ ]:
def evaluate_fixed_resolution_cv(
    labels_path: str,
    comm_df: pd.DataFrame,
    label_col: str = "name",
    n_splits: int = 5,
    random_state: int = 42
):

    if not os.path.exists(labels_path):
        raise FileNotFoundError(f"Labels not found: {labels_path}")

    labels = (pd.read_csv(labels_path)
                .dropna(subset=['account_id', label_col])
                .drop_duplicates(subset=['account_id'])
                .rename(columns={label_col: 'name'}))

    try:
        labels['account_id'] = labels['account_id'].astype(int)
        comm_cast = comm_df.copy()
        comm_cast['account_id'] = comm_cast['account_id'].astype(int)
    except Exception:
        labels['account_id'] = labels['account_id'].astype(str)
        comm_cast = comm_df.copy()
        comm_cast['account_id'] = comm_cast['account_id'].astype(str)

    joined = labels.merge(comm_cast, on='account_id', how='inner')

    n_labeled = len(labels)
    n_joined  = len(joined)
    coverage  = (n_joined / n_labeled) if n_labeled else 0.0
    
    le = LabelEncoder()
    y_all = le.fit_transform(joined['name'].values)
    X_ids = joined['account_id'].values

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_ids, y_all), start=1):
        df_tr = joined.iloc[tr_idx].copy()
        df_te = joined.iloc[te_idx].copy()

        y_true_tr = le.transform(df_tr['name'])
        y_pred_tr = df_tr['community'].values
        nmi_tr = NMI(y_true_tr, y_pred_tr)
        ari_tr = ARI(y_true_tr, y_pred_tr)
        purity_tr = overall_purity(df_tr[['community', 'name']])

        y_true_te = le.transform(df_te['name'])
        y_pred_te = df_te['community'].values
        nmi_te = NMI(y_true_te, y_pred_te)
        ari_te = ARI(y_true_te, y_pred_te)
        purity_te = overall_purity(df_te[['community', 'name']])

        rows.append({
            'fold': fold,
            'n_train': len(df_tr),
            'n_test': len(df_te),
            'train_frac': len(df_tr) / len(joined),
            'NMI_train': nmi_tr,
            'ARI_train': ari_tr,
            'Purity_train': purity_tr,
            'NMI_test': nmi_te,
            'ARI_test': ari_te,
            'Purity_test': purity_te,
        })

    per_fold_df = pd.DataFrame(rows)
    averages = {
        'Avg_NMI_train': float(np.nanmean(per_fold_df['NMI_train'].values)),
        'Avg_ARI_train': float(np.nanmean(per_fold_df['ARI_train'].values)),
        'Avg_Purity_train': float(np.nanmean(per_fold_df['Purity_train'].values)),
        'Avg_NMI_test': float(np.nanmean(per_fold_df['NMI_test'].values)),
        'Avg_ARI_test': float(np.nanmean(per_fold_df['ARI_test'].values)),
        'Avg_Purity_test': float(np.nanmean(per_fold_df['Purity_test'].values)),
        'Avg_train_frac': float(np.nanmean(per_fold_df['train_frac'].values)),
    }

    coverage_info = {
        'n_labeled': int(n_labeled),
        'n_joined': int(n_joined),
        'coverage': float(coverage),
    }

    return per_fold_df, averages, coverage_info

# Transactions 

In [11]:
comm_df = load_communities_fixed_resolution(LPA_COMM_TX_FILE)
print(f"Loaded LPA communities from: {LPA_COMM_TX_FILE}")
print(f"Rows: {len(comm_df):,}  |  Unique accounts: {comm_df['account_id'].nunique():,}")

Loaded LPA communities from: transaction/lpa_tx_lcc_lpa_communities.csv
Rows: 206,451  |  Unique accounts: 206,451


In [ ]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_resolution_cv(
    labels_path=NORM_LABELS_PATH,
    comm_df=comm_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

print("NORMALIZED labels: per-fold metrics (TEST)")
display(norm_per_fold)

print("\nNORMALIZED labels: averages (TEST)")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")


=== NORMALIZED labels: per-fold metrics (TEST) ===


/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,317,80,0.798489,0.404646,0.002887,0.839117,0.437652,0.014436,0.862500
1,2,317,80,0.798489,0.417240,0.011707,0.839117,0.425380,-0.004785,0.850000
2,3,318,79,0.801008,0.422383,0.015038,0.842767,0.392530,-0.035387,0.835443
3,4,318,79,0.801008,0.412045,0.008304,0.842767,0.432684,0.000663,0.873418
4,5,318,79,0.801008,0.403426,-0.000950,0.842767,0.455187,0.049314,0.848101



=== NORMALIZED labels: averages (TEST) ===
Avg_NMI_train: 0.411948
Avg_ARI_train: 0.007397
Avg_Purity_train: 0.841307
Avg_NMI_test: 0.428687
Avg_ARI_test: 0.004848
Avg_Purity_test: 0.853892
Avg_train_frac: 0.800000
Coverage: 4.90%  (397/8102)


In [ ]:
TX_PATH = 'transaction/cross-validation'
os.makedirs(TX_PATH, exist_ok=True) 


norm_per_fold.to_csv(f"{TX_PATH}/lpa_cv_results_norm_per_fold.csv", index=False)

pd.DataFrame([{**norm_avg, **norm_cov}]).to_csv(f"{TX_PATH}/lpa_cv_results_norm_summary.csv", index=False) 

print("\nSaved to:",
      f"{TX_PATH}/lpa_cv_results_norm_per_fold.csv,",
      f"{TX_PATH}/lpa_cv_results_norm_summary.csv")


Saved: transaction/cross-validation/lpa_cv_results_norm_per_fold.csv, transaction/cross-validation/lpa_cv_results_norm_summary.csv


# Trustlines

In [24]:
comm_df = load_communities_fixed_resolution(LPA_COMM_TL_FILE)
print(f"Loaded LPA communities from: {LPA_COMM_TL_FILE}")
print(f"Rows: {len(comm_df):,}  |  Unique accounts: {comm_df['account_id'].nunique():,}")

Loaded LPA communities from: trustline/lpa_trust_lcc_lpa_communities.csv
Rows: 24,586  |  Unique accounts: 24,586


In [ ]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_resolution_cv(
    labels_path=NORM_LABELS_PATH,
    comm_df=comm_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

print("NORMALIZED labels: per-fold metrics (TEST)")
display(norm_per_fold)

print("\nNORMALIZED labels: averages (TEST)")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")


=== NORMALIZED labels: per-fold metrics (TEST) ===


/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,111,28,0.798561,0.292079,0.034444,0.927928,0.347040,0.032960,0.964286
1,2,111,28,0.798561,0.268608,0.022403,0.909910,0.408026,0.079332,1.000000
2,3,111,28,0.798561,0.298500,0.029655,0.927928,0.352985,0.034628,0.964286
3,4,111,28,0.798561,0.310467,0.039347,0.927928,0.294781,0.007786,0.928571
4,5,112,27,0.805755,0.306332,0.032964,0.910714,0.188453,0.014390,0.962963



=== NORMALIZED labels: averages (TEST) ===
Avg_NMI_train: 0.295197
Avg_ARI_train: 0.031762
Avg_Purity_train: 0.920882
Avg_NMI_test: 0.318257
Avg_ARI_test: 0.033819
Avg_Purity_test: 0.964021
Avg_train_frac: 0.800000
Coverage: 1.72%  (139/8102)


In [ ]:
TL_PATH = 'trustline/cross-validation'
os.makedirs(TL_PATH, exist_ok=True) 


norm_per_fold.to_csv(f"{TL_PATH}/lpa_cv_results_norm_per_fold.csv", index=False)


pd.DataFrame([{**norm_avg, **norm_cov}]).to_csv(f"{TL_PATH}/lpa_cv_results_norm_summary.csv", index=False) 

print("\nSaved to:",
      f"{TL_PATH}/lpa_cv_results_norm_per_fold.csv,",
      f"{TL_PATH}/lpa_cv_results_norm_summary.csv")


Saved: trustline/cross-validation/lpa_cv_results_norm_per_fold.csv, trustline/cross-validation/lpa_cv_results_norm_summary.csv
